# 训练模型

In [3]:
import wave
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout


Using TensorFlow backend.


# 核心函数：音频处理函数

在上一个图像识别的案例中，我们使用CV2和PIL库处理图片，现在，我们使用wave库处理音频文件。

get_wavedata函数能够读取音频文件，并转为我们需要的格式。

In [16]:
def get_wavedata(wav_path):
    f = wave.open(wav_path,'rb')
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    #读取音频，转换成int格式
    strData = f.readframes(nframes)
    waveData = np.fromstring(strData,dtype=np.int16)
    #数据归一化
    waveData = waveData*1.0/(max(abs(waveData)))
    waveData = np.reshape(waveData,[nframes,nchannels]).T
    f.close()
    # 处理音频时长
    data = list(np.array(waveData[0]))
    while len(data)>16000:
        del data[len(waveData[0])-1]
        del data[0]
    while len(data)<16000:
        data.append(0)
    data=np.array(data)
    # 平方之后，开平方，取正数，值的范围在  0-1  之间
    data = (data ** 2)** 0.5

    return data

# 数据处理

加载数据集和标签

In [19]:
num_class = 0 # 存放语音总类别的数量
labsIndName=[]      # 训练集标签的名字   
wavs=[] # 训练wav文件集
labels=[] # labels 和 testlabels 这里面存的值都是对应标签的下标，下标对应的名字在labsInd中
testwavs=[] # 测试wav文件集
testlabels=[] # 测试集标签
percent=10   #抽取作测试集的比例，10为抽取10%文件作为测试集

path="data"
dirs = os.listdir(path) # 获取的是目录列表
for i in dirs:
    print("开始加载:",i)
    labsIndName.append(i) # 当前分类进入到标签的名字集
    wavs_path=path+"\\"+i
    testNum=0 #  目前文件夹中进入了测试集的数量
    files = os.listdir(wavs_path) # 某个目录下文件的列表
    #直接用代码区分数据，从文件夹中抽取一定量的文件作为训练集和测试集
    for j in files:
        try:
            waveData = get_wavedata(wavs_path+"\\"+j)
            if testNum < (len(files)/percent) :     #每个文件夹中抽出percent%作为测试集  
                testwavs.append(waveData)
                testlabels.append(labsIndName.index(i))
                testNum+=1
            else:
                wavs.append(waveData)
                labels.append(labsIndName.index(i))
        except:
            pass
num_class = len(labsIndName)

wavs=np.array(wavs)
labels=np.array(labels)
testwavs=np.array(testwavs)
testlabels=np.array(testlabels)

# 标签转换为独热码
labels = keras.utils.to_categorical(labels, num_class)
testlabels = keras.utils.to_categorical(testlabels, num_class)

print(wavs.shape)
print(labels.shape)
print(testwavs.shape)
print(testlabels.shape)


开始加载: 0zero
开始加载: 1one
(13, 16000)
(13, 2)
(2, 16000)
(2, 2)


e:\application\python\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


# 模型训练

In [ ]:

model = Sequential()
model.add(Dense(1024, activation='relu',input_shape=(16000,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_class, activation='softmax'))

# 模型训练

In [ ]:
# 将识别准确率作为模型评估标准
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(wavs, labels, batch_size=100, epochs=20, verbose=2, validation_data=(testwavs, testlabels))

# 评估模型效果
score = model.evaluate(testwavs, testlabels, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1]) 

model.save('0to9_model.h5')

# 应用模型

In [ ]:
import wave
import numpy as np
import os
from keras.models import load_model
wavs=[]

# 加载模型
model = load_model('0to9_model.h5') # 加载训练模型

wavs.append(get_wavedata("testdata/test.wav"))
X=np.array(wavs)
print(X.shape)
result=model.predict(X[0:1])[0] 
print("识别结果",result)
#标签集的名字
name = ["0","1","2","3","4","5","6","7","8","9"] # 创建一个跟训练时一样的标签集
ind=0 # 结果中最大的一个数
for i in range(len(result)):
    if result[i] > result[ind]:
        ind=i
print("识别的语音结果是：",name[ind])
